In [2]:
import pandas as pd
import altair as alt
from vega_datasets import data
# from geopy.geocoders import Nominatim

url = "https://github.com/SanderP99/Badminton-Data/blob/main/out/tournaments.csv"
source = pd.read_csv('data/raw.txt')
worldCities = pd.read_excel('data/worldcities.xlsx')

In [3]:
# geolocator = Nominatim(user_agent="myApp")
lons = []
lats = []
duplicates = {}
size = []
count = []
year = []
continent = []

# for city in source['city']:
#     location=geolocator.geocode(city)
#     lons.append(location.longitude)
#     lats.append(location.latitude)
#     if city in duplicates:
#         duplicates[city] = duplicates[city] + 1
#     else:
#         duplicates[city] = 1

for city in source['city']:
    lonQuery = worldCities.loc[worldCities['city_ascii'] == city, 'lng']
    latQuery = worldCities.loc[worldCities['city_ascii'] == city, 'lat']
    if (lonQuery.empty):
        country = source.loc[source['city'] == city, 'country'].iloc[0]
        lonQuery = worldCities.loc[worldCities['country'] == country, 'lng']
        latQuery = worldCities.loc[worldCities['country'] == country, 'lat']
    if (lonQuery.empty):
        continue
    lons.append(lonQuery.iloc[0])
    lats.append(latQuery.iloc[0])
    if city in duplicates:
        duplicates[city] = duplicates[city] + 1
    else:
        duplicates[city] = 1

import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

for country in source['country']:
    if (country=='Korea'):
        continent.append(country_to_continent('Korea, Republic of'))
    else:
        continent.append(country_to_continent(country))

for city in source['city']:
    count.append(duplicates[city])
    size.append(10*duplicates[city])

for date in source['start_date']:
    y = ""
    for i in range(len(date) - 3):
        num = True
        for j in range(4):
            num = num & date[i + j].isdigit()
        if num :
            y = ""
            for j in range(4):
                y += date[i + j]
    year.append(y)

source['longitude'] = lons
source['latitude'] = lats
source['size'] = size
source['count'] = count
source['year'] = year
source['continent'] = continent

In [4]:
source.sample(5)

,name,organization,city,country,type,start_date,end_date,link,longitude,latitude,size,count,year,continent
60,VICTOR Oceania Junior Mixed Team Championships...,Badminton Oceania,Auckland,New Zealand,Continental Junior Team Championships,2/10/2023,2/12/2023,/sport/tournament?id=C6F94211-0234-4C35-9889-D...,174.7400,-36.8406,60,6,2023,Oceania
63,VICTOR Oceania Junior Championships 2023,Badminton Oceania,Auckland,New Zealand,Continental Junior Individual Championships,2/13/2023,2/16/2023,/sport/tournament?id=10FA45E0-4CCE-4837-9404-7...,174.7400,-36.8406,60,6,2023,Oceania
25,2023 European Mixed Team Championships - Quali...,Badminton Europe,Glasgow,United Kingdom,Continental Team Championships,12/15/2022,12/18/2022,/sport/tournament?id=FF18B111-F177-4493-92ED-D...,-4.2500,55.8611,10,1,2022,Europe
68,VICTOR Oceania Mixed Team Championships 2023,Badminton Oceania,Auckland,New Zealand,Continental Team Championships,2/17/2023,2/19/2023,/sport/tournament?id=081B7F90-BD20-45B6-A6FD-0...,174.7400,-36.8406,60,6,2023,Oceania
13,HSBC BWF World Tour Finals 2022,Badminton World Federation,Bangkok,Thailand,HSBC BWF World Tour Finals,12/7/2022,12/11/2022,/sport/tournament?id=C99D04D4-0B11-416E-9F7B-3...,100.4942,13.7525,20,2,2022,Asia


In [6]:
countries_url = data.world_110m.url
countries = alt.topo_feature(countries_url, "countries")
countries_base = alt.Chart(countries).mark_geoshape(
fill='lightgray',
stroke='white'
).project(
    "mercator"
).transform_filter(
    alt.datum.id!=10
)

value_1 = list(source["country"].unique())
value_2 = list(source["continent"].unique())
value_3 = list(source["year"].unique())

option_1 = [None] + value_1
option_2 = [None] + value_2
option_3 = [None] + value_3

label_1 = ["All"] + value_1
label_2 = ["All"] + value_2
label_3 = ["All"] + value_3

dropdown1 = alt.binding_select(options=option_1, labels=label_1, name="country")
selection1 = alt.selection_single(fields=['country'], bind=dropdown1, name='country')

dropdown2 = alt.binding_select(options=option_2, labels=label_2, name="continent")
selection2 = alt.selection_single(fields=['continent'], bind=dropdown2, name='continent')

dropdown3 = alt.binding_select(options=option_3, labels=label_3, name="year")
selection3 = alt.selection_single(fields=['year'], bind=dropdown3, name='year')

op_condition = {"condition":
                {"test":
                 {"and":
                  [{"selection": "year"},
                   {"selection": "continent"}]},
                    "value": 1},
                "value": 0.1}

tournaments = alt.Chart(source).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size = 'count:Q',
    tooltip=[alt.Tooltip('name:N', title='Name'),
             alt.Tooltip('organization:N', title='Organization'),
             alt.Tooltip('city:N', title='City'),
             alt.Tooltip('country:N', title='Country'),
             alt.Tooltip('type:N', title='Type'),
             alt.Tooltip('count:Q', title='Number of tournaments held in this city'),
             alt.Tooltip('start_date:T', title='Start date of the latest tournament'),
             alt.Tooltip('end_date:T', title='End date of the latest tournament')],
    opacity=op_condition
).add_selection(
    selection2, selection3
)
(countries_base+tournaments).properties(title='Badminton Tournament around the World (Nov.2022~Mar.2023)', width=900, height=500)

alt.LayerChart(...)